In [1]:
# !pip install langchain neo4j openai wikipedia tiktoken langchain_openai

In [2]:
from langchain.graphs import Neo4jGraph

url = "neo4j+s://d6b3533f.databases.neo4j.io"
username ="neo4j"
password = "8"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

# Delete the graph
graph.query("MATCH (n) DETACH DELETE n")

[]

In [3]:
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [4]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

In [5]:
import os
from langchain.chains.openai_functions import (
    create_structured_output_runnable,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

os.environ["OPENAI_API_KEY"] = "sk-"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    lang: Optional[str] = "Vietnamese",
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
- **Language**: The document is in {lang}, so the node ids, node labels, attributes and relationships MUST also be in {lang}.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text, but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
If the entity is previously identified in the list below, use the identifier in the list instead. For example, if there is a "Bể dầu khí sông Hồng"
in the list, and there is a "Bể Sông Hồng" mentioned, use "Bể dầu khí sông Hồng" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
{'- **Identified entities:** {identified_id_str}'}
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_runnable(KnowledgeGraph, llm, prompt,)

You can see that we are using the 16k version of the GPT-3.5 model. The main reason is that the OpenAI function output is a structured JSON object, and structured JSON syntax adds a lot of token overhead to the result. Essentially, you are paying for the convenience of structured output in increased token space.

Besides the general instructions, I have also added the option to limit which node or relationship types should be extracted from text. You'll see through examples why this might come in handy.
We have the Neo4j connection and LLM prompt ready, which means we can define the information extraction pipeline as a single function.

In [6]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    identified_ids: Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    identified_ids_str = ", ".join(identified_ids) if identified_ids else ""
    extract_chain = get_extraction_chain(nodes, allowed_rels=rels)
    data = extract_chain.invoke({"input": document.page_content, "identified_id_str": identified_ids_str})
    print(data)
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [7]:
from docx import Document as Docx

def read_docx(file_path):
    doc = Docx(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)

file_path = 'Resources\SongHongDoc.docx'
text = read_docx(file_path)
raw_document = Document(page_content=text)
# extract_and_store_graph(document)

In [8]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=128)

# Only take the first the raw_documents
documents = text_splitter.split_documents([raw_document])
from tqdm import tqdm
entities = []
for i, d in tqdm(enumerate(documents), total=len(documents)):
    extract_and_store_graph(d, identified_ids=entities)
    result = graph.query("MATCH (n) RETURN n")
    entities = [str(record['n']) for record in result]

  0%|          | 0/19 [00:00<?, ?it/s]

nodes=[Node(id='Bể trầm tích Sông Hồng', type='bể trầm tích', properties=[Property(key='diện tích', value='lớn nhất trong số các bể trầm tích Kainozoi chứa dầu khí trên thềm lục địa Việt Nam'), Property(key='công ty quan tâm', value='Tập đoàn Dầu khí Việt Nam và nhiều nhà thầu dầu khí nước ngoài')]), Node(id='Hàm Rồng', type='phát hiện dầu khí', properties=[Property(key='đối tượng đá móng', value='carbonat')]), Node(id='Hàm Rồng Đông', type='phát hiện dầu khí', properties=[Property(key='đối tượng đá móng', value='carbonat')]), Node(id='Hàm Rồng Nam', type='phát hiện dầu khí', properties=[Property(key='đối tượng đá móng', value='carbonat')]), Node(id='Kì Lân', type='phát hiện dầu khí', properties=[Property(key='vị trí', value='phía Bắc Bể')]), Node(id='Cá Voi Xanh', type='phát hiện dầu khí', properties=[Property(key='vị trí', value='phía Nam bể'), Property(key='đối tượng đá móng', value='carbonat Miocen')]), Node(id='Kèn Bầu', type='phát hiện khí', properties=[Property(key='vị trí', val

  5%|▌         | 1/19 [00:13<03:58, 13.24s/it]

nodes=[Node(id='bể Sông Hồng', type='bể', properties=[]), Node(id='Viện Dầu Khí', type='viện', properties=[])] rels=[Relationship(source=Node(id='bể Sông Hồng', type='bể'), target=Node(id='Viện Dầu Khí', type='viện'), type='đã đặt hàng', properties=[Property(key='nhiệm vụ', value='Tổng hợp, hệ thống hóa và đánh giá các kết quả nghiên cứu tiềm năng dầu khí đã thực hiện ở bể Sông Hồng')])]


 11%|█         | 2/19 [00:16<02:09,  7.64s/it]

...

nodes=[Node(id='PVN', type='Tổng Công Ty Dầu Khí Việt Nam', properties=[]), Node(id='PCOSB', type='Công Ty Dịch Vụ Khoan Dầu Khí', properties=[]), Node(id='VN-TQ', type='Việt Nam - Trung Quốc', properties=[]), Node(id='PIDC', type='Công Ty Khai Thác Dầu Khí Phú Quốc', properties=[]), Node(id='PVEP', type='Công Ty Khai Thác Dầu Khí Phương Đông', properties=[]), Node(id='VGP', type='Công Ty Khai Thác Dầu Khí Việt - Nga', properties=[]), Node(id='ExxonMobil', type='ExxonMobil', properties=[]), Node(id='BP89', type='BP', properties=[]), Node(id='BP91', type='BP', properties=[])] rels=[Relationship(source=Node(id='PVN', type='Tổng Công Ty Dầu Khí Việt Nam'), target=Node(id='PCOSB', type='Công Ty Dịch Vụ Khoan Dầu Khí'), type='thực hiện khảo sát'), Relationship(source=Node(id='PVN', type='Tổng Công Ty Dầu Khí Việt Nam'), target=Node(id='VN-TQ', type='Việt Nam - Trung Quốc'), type='thỏa thuận thăm dò chung'), Relationship(source=Node(id='PIDC', type='Công Ty Khai Thác Dầu Khí Phú Quốc'), targ

 37%|███▋      | 7/19 [01:30<03:07, 15.63s/it]

nodes=[Node(id='Bể Sông Hồng', type='Bể trầm tích', properties=[Property(key='diện tích', value='220.000km2'), Property(key='độ sâu nước biển', value='20 - 40m (phía Bắc), 20-90m (vùng trung tâm), 30-800m (vùng phía Nam)'), Property(key='hình dạng', value='hình thoi')]), Node(id='Công Tác Thăm Dò Địa Chấn', type='Công tác thăm dò', properties=[Property(key='lịch sử', value='Trước năm 1980 hầu như chưa có hoạt động tìm kiếm thăm dò ở ngoài khơi bể Sông Hồng ngoại trừ các lô từ 117-121 ở phía Nam bể Sông Hồng. Năm 1974 chính quyền Sài Gòn cũ cho công ty dầu khí quốc tế tiến hành tìm kiếm thăm dò dầu khí ở vùng ngoài khơi Nam Trung Bộ, Việt Nam. Hai mạng lưới địa chấn khu vực bao gồm: WA74-pkb và WA-74-shv đã được tiến hành thu nổ từ vùng biển miền Trung, Ninh Thuận tới các đảo Hoàng Tử Anh, Hoàng Tử Em khu vực quần đảo Hoàng Sa. Các khảo sát địa chấn 2D này có mạng lưới rất thưa, 35 x 40km và 40 x 60km. Công việc khảo sát bị chấm dứt khi miền Nam được giải phóng, đất nước ta thống nhất n

 42%|████▏     | 8/19 [02:20<03:12, 17.53s/it]


KeyboardInterrupt: 

In [ ]:
result = graph.query("MATCH (n) RETURN n")
entities = [str(record['n']) for record in result]
entities

["{'côngTyQuanTâm': 'Tập đoàn Dầu khí Việt Nam và nhiều nhà thầu dầu khí nước ngoài', 'name': 'Bể Trầm Tích Sông Hồng', 'diệnTích': 'lớn nhất trong số các bể trầm tích Kainozoi chứa dầu khí trên thềm lục địa Việt Nam', 'id': 'Bể Trầm Tích Sông Hồng'}",
 "{'name': 'Hàm Rồng', 'id': 'Hàm Rồng', 'đốiTượngĐáMóng': 'carbonat'}",
 "{'name': 'Hàm Rồng Đông', 'id': 'Hàm Rồng Đông', 'đốiTượngĐáMóng': 'carbonat'}",
 "{'name': 'Hàm Rồng Nam', 'id': 'Hàm Rồng Nam', 'đốiTượngĐáMóng': 'carbonat'}",
 "{'vịTrí': 'phía Bắc Bể', 'name': 'Kì Lân', 'id': 'Kì Lân'}",
 "{'vịTrí': 'phía Nam bể', 'name': 'Cá Voi Xanh', 'id': 'Cá Voi Xanh', 'đốiTượngĐáMóng': 'carbonat Miocen'}",
 "{'vịTrí': 'phần trung tâm bể', 'name': 'Kèn Bầu', 'id': 'Kèn Bầu', 'đốiTượngBẫyĐịaTầng': 'tuổi Miocen muộn'}"]